In [1]:
import os
import trimesh as tm 
from time import time, process_time
import csv
from constants import data_dir, csv_dir, dimension
from tqdm import tqdm
import numpy as np
import trimesh

In [2]:
start = 1
end = 100
make_header = True

csv_path = os.path.join(csv_dir, "sub_results", "iou_distance_{}_{}.csv".format(start, end))
n = end - start

In [3]:
start_time = process_time()
actual_start_time = time()

obj_list_full = sorted(os.listdir(data_dir))
if end == -1 or end > len(obj_list_full):
    end = len(obj_list_full)
obj_list = obj_list_full[0:end]

In [4]:
def write_header(fieldnames):
    with open(csv_path, 'wt', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()


def write_row(row):
    with open(csv_path, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)


def print_csv():
    with open(csv_path, newline='') as f:
        reader = csv.reader(f)
        for row in reader:
            print(row)

In [5]:
if make_header:
    write_header([''] + obj_list_full)
    
del obj_list_full

In [8]:
def iou(voxel_matrix1, voxel_matrix2):
    return np.count_nonzero(np.logical_and(voxel_matrix1, voxel_matrix2)) / np.count_nonzero(np.logical_or(voxel_matrix1, voxel_matrix2))

In [ ]:
data = np.zeros(n*(n-1)//2)
ii = np.zeros(n*(n-1)//2, dtype=int)
jj = np.zeros(n*(n-1)//2, dtype=int)

In [6]:
for idx1, obj1_name in enumerate(tqdm(obj_list[start:], position=0, leave=True), start=start):
    voxel_matrix1 = np.load(os.path.join(data_dir, obj1_name, "{}.npy".format(obj1_name)), allow_pickle=True)
    try:
        assert(voxel_matrix1.shape == (dimension, dimension, dimension))
    except:
        print(obj1_name, "has shape", voxel_matrix1.shape, "instead of", (dimension, dimension, dimension))
        break
        continue
    print(voxel_matrix1)
    break
    for idx2, obj2_name in enumerate(obj_list[:idx1-1]):
        data(idx1 + idx2) = iou(voxel_matrix1, voxel_matrix1)
        
        
        

  0%|          | 0/100 [00:00<?, ?it/s]

00000002 has shape () instead of (128, 128, 128)


In [7]:
voxel_matrix1

array(<trimesh.VoxelGrid(128, 47, 128)>, dtype=object)

In [ ]:
for i in range(128):
    for j in range(47):
        for k in range(128):
            print(voxel_matrix1[i,j,k])

In [ ]:
def get_obj(obj_name):
    obj_dir = os.path.join(data_dir, obj_name)
    obj_path = os.path.join(obj_dir, os.listdir(obj_dir)[0])
    return o3d.io.read_triangle_mesh(obj_path)

In [ ]:
mesh = get_obj('00000002')

In [ ]:
o3d.visualization.draw_geometries([mesh])

In [ ]:
import os

In [ ]:
os.environ["DISPLAY"] = "0.0"

In [ ]:
import meshplot as mp

In [ ]:
total_count = end-start
print_step = total_count // 10
last_time = process_time()
for idx1, pc1_name in enumerate(tqdm(pc_list[start:], position=0, leave=True)):
    num_obj = idx1 + start
    row = [None] * (num_obj + 1)
    row[0] = pc1_name
    pc1 = get_pc(pc1_name)
    for idx2 in range(num_obj):
        pc2 = get_obj(pc_list[idx2])
        chamfer = iou_distance(obj1, pc2)[0].item()  # calculate chamfer distance
        row[idx2 + 1] = chamfer
    # write to csv
    write_row(row)
    if idx1 % print_step == 0:
        current_time = process_time()
        print(idx1, "of", total_count, "finished.")
        print("total time:", current_time - start_time)
        print("time for last", print_step, "iterations:", current_time - last_time)
        last_time = current_time

current_time = process_time()
print("finished! total time for all:", current_time - start_time)
print("time for last few iterations:", current_time - last_time)
print("total time for {} iterations: {}".format(end-start, time()-actual_start_time))